In [1]:
%matplotlib inline
from __future__ import division
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.dummy import DummyClassifier
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn import preprocessing
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import processData
from settings import *
from mappings import *
from datanetwork import *
from bluetoothdata import *
from features import *

In [3]:
survey_df, sms_call_df, communication_time_graphs = processData.processData()

Loading communication data...
Fixing data format...
Shape of sms-call data: (537575, 5)
Dropping instances where sender and receiver match...
Shape of sms-call data: (533121, 5)
             DateTime  SenderID  ReceiverID  EventType  EventLength
0 2011-08-15 20:03:31     20905       13116          0           15
1 2011-08-15 20:03:37     20905       13116          0            0
2 2011-08-15 20:47:40     75178       30952          0           39
3 2011-08-15 20:47:50     75178       30952          0           23
4 2011-08-15 20:48:52     75178       60830          0           14

Loading survey data...
Fixing data format...
Shape of survey dataframe (199, 59)
   egoid  age_1  hometown_1               ethnicity_1 gender_1  \
0  97900   18.0         6.0           White/Caucasian   Female   
1  97942   19.0         6.0           White/Caucasian     Male   
2  31583   18.0         5.0           White/Caucasian   Female   
3  10281   18.0         6.0  Mexican American/Chicano     Male   
4 

/Users/aastha/anaconda2/lib/python2.7/site-packages/pandas/core/generic.py:3855: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  regex=regex)


Before:


/Users/aastha/anaconda2/lib/python2.7/site-packages/pandas/core/generic.py:3660: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
/Users/aastha/anaconda2/lib/python2.7/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s



Extracting corresponding communication network...
Number of interactions amongst selected users: 176317
Interactions date range: 2011-08-15 20:47:40 - 2014-08-22 00:23:55

Building network...
Study period: 1
Making network from: 2011-08-15 20:47:40 to 2012-01-28 12:19:54
Range: >= 2011-08-01 00:00:00 and < 2012-01-01 00:00:00
Number of interactions: 28251

Study period: 2
Making network from: 2012-01-28 12:19:54 to 2012-05-09 09:10:55
Range: >= 2012-01-01 00:00:00 and < 2012-05-01 00:00:00
Number of interactions: 30017

Study period: 3
Making network from: 2012-05-09 09:10:55 to 2012-08-20 10:16:29
Range: >= 2012-05-01 00:00:00 and < 2012-08-01 00:00:00
Number of interactions: 9917

Study period: 4
Making network from: 2012-08-20 10:16:29 to 2013-01-18 21:25:53
Range: >= 2012-08-01 00:00:00 and < 2013-01-01 00:00:00
Number of interactions: 34730

Study period: 5
Making network from: 2013-01-18 21:25:53 to 2013-05-02 09:01:05
Range: >= 2013-01-01 00:00:00 and < 2013-05-01 00:00:00
Numb

In [4]:
bluetooth_time_graphs, weekend_bluetooth_time_graphs = getBluetoothNetworks(survey_df)

In [5]:
def getTrainTestPairs(time_graphs):
    train_pairs = list()
    print "Making Train"
    for t in survey_number[:-2]:
        g = time_graphs[t]
        print 't =', t, '-> t =', t+1
        for eid in g.edges():
            uid1 = eid[0]
            uid2 = eid[1]
            instance = [t, uid1, uid2]
            train_pairs.append(instance)
    
    test_pairs = list()
    print "Making Test"
    for t in survey_number[-2:-1]:
        g = time_graphs[t]
        print 't =', t, '-> t =', t+1
        for eid in g.edges():
            uid1 = eid[0]
            uid2 = eid[1]
            instance = [t, uid1, uid2]
            test_pairs.append(instance)

    train_pairs = pd.DataFrame(train_pairs, columns=['time', 'uid1', 'uid2'])
    test_pairs = pd.DataFrame(test_pairs, columns=['time', 'uid1', 'uid2'])
    
    return train_pairs, test_pairs

In [6]:
def computeFeatures(g, i, j):
    f1 = getAttributeSim(g, i, j)
    f2 = getTriadWeights(g, i, j)
    f3 = getbeliefSimilarity(g, i, j)
    f4,f5 = social_tie_persistence(g, i, j)
    features = [f4, f1, f2, f3, f5]
    return features

def combinedFeatures(g, bg, i, j):
    #common
    f1 = getAttributeSim(g, i, j)
    f2 = getbeliefSimilarity(g, i, j)    
    
    #communication network
    f3 = getTriadWeights(g, i, j)
    f4,f5 = social_tie_persistence(g, i, j)
    
    #bluetooth network
    f6 = getTriadWeights(bg, i, j)
    f7,f8 = social_tie_persistence(bg, i, j)
    
    features = [f4,f1,f2,f3,f5,f6,f7,f8]
    return features

In [7]:
##
#1: just communication
#2: just bluetooth
#3: combined
def computePairTrainTest(pairs, bluetooth_time_graphs, communication_time_graphs, weighted, whichNetwork):
    print "weighted:", weighted
    X = list()
    y = list()    
    for ix, row in pairs.iterrows():
        t = row['time']
        uid1 = row['uid1']
        uid2 = row['uid2']
        
        g = communication_time_graphs[t]
        bg = bluetooth_time_graphs[t]
        
        next_g = communication_time_graphs[t+1]
                
        if(whichNetwork == 1):
            features = computeFeatures(g, uid1, uid2)
        elif(whichNetwork == 2):
            features = computeFeatures(bg, uid1, uid2)
        elif(whichNetwork == 3):
            features = combinedFeatures(g, bg, uid1, uid2)
        
        X.append(features)
        
        if next_g.has_edge(uid1, uid2):
            if weighted == True:
                target = getWeight(next_g[uid1][uid2]['weight'], weight_type)
            else:
                target = getWeight(next_g[uid1][uid2]['weight'], 'binary')
        else:
            target = 0
        y.append(target)
    return X, y
        
def makePairTrainTest(train_pairs, test_pairs, bluetooth_time_graphs, communication_time_graphs, weighted, whichNetwork):
    X_train, y_train = computePairTrainTest(train_pairs, bluetooth_time_graphs, communication_time_graphs, weighted, whichNetwork)
    X_test, y_test = computePairTrainTest(test_pairs, bluetooth_time_graphs, communication_time_graphs, weighted, whichNetwork)
    
    X_train = np.array(X_train)
    y_train = np.array(y_train)

    X_test = np.array(X_test)
    y_test = np.array(y_test)

    print "Train data shape", X_train.shape, y_train.shape
    print "Test data shape", X_test.shape, y_test.shape
    
    return X_train, y_train, X_test, y_test

def makeData(X_train, y_train, X_test, y_test):
    #make column names
    features_names = list()
    for ix in range(X_train.shape[1]):
        features_names.append('f'+str(ix+1))

    target_names = list()
    if y_train.ndim == 1:
        target_names = ['y']
    else:
        for ix in range(y_train.shape[1]):
            target_names.append('y'+str(ix+1))

    column_names = features_names + target_names

    #turn lists into dataframes
#    print "Train Data"
    X_train = pd.DataFrame(X_train)
    y_train = pd.DataFrame(y_train)
    train_df = pd.concat([X_train,y_train],axis=1)
    train_df.columns = column_names
#    print train_df.head()

#    print "Test Data"
    X_test = pd.DataFrame(X_test)
    y_test = pd.DataFrame(y_test)
    test_df = pd.concat([X_test,y_test],axis=1)
    test_df.columns = column_names
#    print test_df.head()
    
#     #Drop duplicates if any
#     print "Check for duplicates in train data"
#     print "Train shape before:", train_df.shape
    train_df = pd.DataFrame.drop_duplicates(pd.DataFrame(train_df))
#     print "Train shape after:", train_df.shape
#     print ''

#     print "Test shape before:", test_df.shape
    test_df = pd.DataFrame.drop_duplicates(pd.DataFrame(test_df))
#     print "Test shape after:", test_df.shape
    
    # perform min max normalization on the features
#     print "Min Max Normalization..."
#     scaler = preprocessing.MinMaxScaler()
#     train_df[features_names] = scaler.fit_transform(train_df[features_names])
#     test_df[features_names] = scaler.transform(test_df[features_names])

    return train_df, test_df, features_names, target_names

In [8]:
def fitWeightModel(train_x, train_y, test_x, test_y):
    clf =  LinearRegression()
    clf.fit(train_x, train_y)
    y_hat = clf.predict(test_x)
    mse = metrics.mean_squared_error(test_y, y_hat)
    print 'MSE %0.2f'% mse
    return y_hat

def fitClassModel(train_x, train_y, test_x, test_y):
    clf =  LogisticRegression(class_weight="balanced", random_state=0)
    clf.fit(train_x, train_y)
    y_hat = clf.predict(test_x)
    f1 = metrics.f1_score(test_y, y_hat, average='weighted')
    acc = metrics.accuracy_score(test_y, y_hat)
    cm = metrics.confusion_matrix(test_y, y_hat)

    #print clf.coef_
    print 'F1 %0.2f' % f1
#    print 'Accuracy %0.2f' % acc
    print "Confusion matrix"
    print cm
    return y_hat

def baselineRegressor(train_x, train_y, test_x, test_y):
    #Majority
    print "Baseline: Mean Regressor"
    clf1 = DummyRegressor (strategy='mean')
    clf1.fit(train_x, train_y)
    yhat1 = clf1.predict(test_x)
    
    mse = metrics.mean_squared_error(test_y, yhat1)
    print 'MSE %0.2f'% mse
    
    #Random
    print "Baseline: Median Regressor"
    clf2 = DummyRegressor (strategy='median')
    clf2.fit(train_x, train_y)
    yhat2 = clf2.predict(test_x)
    
    mse = metrics.mean_squared_error(test_y, yhat2)
    print 'MSE %0.2f'% mse
    
    return yhat1, yhat2

def baselineClassifier(train_x, train_y, test_x, test_y):
#     #Majority
#     print "Baseline: Majority Classifier"
#     clf1 = DummyClassifier(strategy='most_frequent',random_state=0)
#     clf1.fit(train_x, train_y)
#     yhat1 = clf1.predict(test_x)
    
#     f1 = metrics.f1_score(test_y, yhat1, average='weighted')
#     acc = metrics.accuracy_score(test_y, yhat1)
#     cm = metrics.confusion_matrix(test_y, yhat1)

#     print 'F1 %0.2f' % f1
#     print 'Accuracy %0.2f' % acc
#     print "Confusion matrix"
#     print cm
    
    #Random
    print "Baseline: Random Classifier"
    clf2 = DummyClassifier(strategy='uniform',random_state=0)
    clf2.fit(train_x, train_y)
    yhat2 = clf2.predict(test_x)
    
    f1 = metrics.f1_score(test_y, yhat2, average='weighted')
    acc = metrics.accuracy_score(test_y, yhat2)
    cm = metrics.confusion_matrix(test_y, yhat2)

    print 'F1 %0.2f' % f1
#     print 'Accuracy %0.2f' % acc
    print "Confusion matrix"
    print cm
    
    return yhat2

In [9]:
train_pairs, test_pairs = getTrainTestPairs(communication_time_graphs)
print train_pairs.shape
print test_pairs.shape

Making Train
t = 1 -> t = 2
t = 2 -> t = 3
t = 3 -> t = 4
t = 4 -> t = 5
Making Test
t = 5 -> t = 6
(851, 3)
(203, 3)


In [10]:
## Based on Communication network
##
#1: just communication
#2: just bluetooth
#3: combined

#regression
weighted = True
for whichNetwork in [1,2,3]:
    print whichNetwork
    print "Weighted Model:", weighted
    X_train, y_train, X_test, y_test = makePairTrainTest(train_pairs, test_pairs, 
                                                         bluetooth_time_graphs, 
                                                         communication_time_graphs, 
                                                         weighted, whichNetwork)
    train_df, test_df, features_names, target_names = makeData(X_train, y_train, X_test, y_test)
    print features_names, target_names
    
    print train_df.shape, test_df.shape

    print "Min and max edge weight train:", train_df.y.min(),train_df.y.max()
    print "Min and max edge weight test:", test_df.y.min(),test_df.y.max()

    print "Model results:"
    fitWeightModel(train_df[features_names], train_df[target_names], test_df[features_names], test_df[target_names])
            
    print "Yesterday results:"
    fitWeightModel(train_df.f1.values.reshape(-1, 1), train_df[target_names], test_df.f1.values.reshape(-1, 1), test_df[target_names])
    
    print "Baseline results"
    baselineRegressor(train_df[features_names], train_df[target_names], test_df[features_names], test_df[target_names])
    
    print "*"*100

1
Weighted Model: True
weighted: True
weighted: True
Train data shape (851, 5) (851,)
Test data shape (203, 5) (203,)
['f1', 'f2', 'f3', 'f4', 'f5'] ['y']
(833, 6) (194, 6)
Min and max edge weight train: 0.0 8.58204416374
Min and max edge weight test: 0.0 8.82849412947
Model results:
MSE 2.50
Yesterday results:
MSE 2.36
Baseline results
Baseline: Mean Regressor
MSE 4.21
Baseline: Median Regressor
MSE 4.00
****************************************************************************************************
2
Weighted Model: True
weighted: True
weighted: True
Train data shape (851, 5) (851,)
Test data shape (203, 5) (203,)
['f1', 'f2', 'f3', 'f4', 'f5'] ['y']
(798, 6) (152, 6)
Min and max edge weight train: 0.0 8.58204416374
Min and max edge weight test: 0.0 8.82849412947
Model results:
MSE 3.59
Yesterday results:
MSE 4.12
Baseline results
Baseline: Mean Regressor
MSE 4.21
Baseline: Median Regressor
MSE 4.11
*********************************************************************************

In [11]:
weighted = False
for whichNetwork in [1,2,3]:
    print whichNetwork
    print "Weighted Model:", weighted
    X_train, y_train, X_test, y_test = makePairTrainTest(train_pairs, test_pairs, 
                                                         bluetooth_time_graphs, 
                                                         communication_time_graphs, 
                                                         weighted, whichNetwork)
    train_df, test_df, features_names, target_names = makeData(X_train, y_train, X_test, y_test)
    print features_names, target_names
    
    print train_df.shape, test_df.shape
    
    print "Train Distribution:", Counter(train_df.y)
    print "Test Distribution:", Counter(test_df.y)

    print "Model results:"
    fitClassModel(train_df[features_names], train_df[target_names], test_df[features_names], test_df[target_names])

    print "Train Distribution:", Counter(train_df.y)
    print "Test Distribution:", Counter(test_df.y)

    print "Yesterday results:"
    fitClassModel(train_df.f1.values.reshape(-1, 1), train_df[target_names], test_df.f1.values.reshape(-1, 1), test_df[target_names])

    print "Train Distribution:", Counter(train_df.y)
    print "Test Distribution:", Counter(test_df.y)

    print "Baseline results:"
    baselineClassifier(train_df[features_names], train_df.y, test_df[features_names], test_df.y)
    print "*"*100

1
Weighted Model: False
weighted: False
weighted: False
Train data shape (851, 5) (851,)
Test data shape (203, 5) (203,)
['f1', 'f2', 'f3', 'f4', 'f5'] ['y']
(827, 6) (193, 6)
Train Distribution: Counter({1: 534, 0: 293})
Test Distribution: Counter({1: 107, 0: 86})
Model results:
F1 0.76
Confusion matrix
[[65 21]
 [25 82]]
Train Distribution: Counter({1: 534, 0: 293})
Test Distribution: Counter({1: 107, 0: 86})
Yesterday results:
F1 0.75
Confusion matrix
[[63 23]
 [25 82]]
Train Distribution: Counter({1: 534, 0: 293})
Test Distribution: Counter({1: 107, 0: 86})
Baseline results:
Baseline: Random Classifier
F1 0.52
Confusion matrix
[[44 42]
 [50 57]]
****************************************************************************************************
2
Weighted Model: False
weighted: False


/Users/aastha/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


weighted: False
Train data shape (851, 5) (851,)
Test data shape (203, 5) (203,)
['f1', 'f2', 'f3', 'f4', 'f5'] ['y']
(743, 6) (134, 6)
Train Distribution: Counter({1: 471, 0: 272})
Test Distribution: Counter({1: 78, 0: 56})
Model results:
F1 0.61
Confusion matrix
[[31 25]
 [27 51]]
Train Distribution: Counter({1: 471, 0: 272})
Test Distribution: Counter({1: 78, 0: 56})
Yesterday results:
F1 0.46
Confusion matrix
[[29 27]
 [45 33]]
Train Distribution: Counter({1: 471, 0: 272})
Test Distribution: Counter({1: 78, 0: 56})
Baseline results:
Baseline: Random Classifier
F1 0.57
Confusion matrix
[[29 27]
 [31 47]]
****************************************************************************************************
3
Weighted Model: False
weighted: False
weighted: False
Train data shape (851, 8) (851,)
Test data shape (203, 8) (203,)
['f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8'] ['y']
(845, 9) (199, 9)
Train Distribution: Counter({1: 538, 0: 307})
Test Distribution: Counter({1: 109, 0: 90})


In [12]:
## Based on Weekend network
##
#1: just communication
#2: just bluetooth
#3: combined

#regression
weighted = True
for whichNetwork in [1,2,3]:
    print whichNetwork
    print "Weighted Model:", weighted
    X_train, y_train, X_test, y_test = makePairTrainTest(train_pairs, test_pairs, 
                                                         weekend_bluetooth_time_graphs, 
                                                         communication_time_graphs, 
                                                         weighted, whichNetwork)
    train_df, test_df, features_names, target_names = makeData(X_train, y_train, X_test, y_test)
    print features_names, target_names
    
    print train_df.shape, test_df.shape

    print "Min and max edge weight train:", train_df.y.min(),train_df.y.max()
    print "Min and max edge weight test:", test_df.y.min(),test_df.y.max()

    print "Model results:"
    fitWeightModel(train_df[features_names], train_df[target_names], test_df[features_names], test_df[target_names])
            
    print "Yesterday results:"
    fitWeightModel(train_df.f1.values.reshape(-1, 1), train_df[target_names], test_df.f1.values.reshape(-1, 1), test_df[target_names])
    
    print "Baseline results"
    baselineRegressor(train_df[features_names], train_df[target_names], test_df[features_names], test_df[target_names])
    
    print "*"*100

1
Weighted Model: True
weighted: True
weighted: True
Train data shape (851, 5) (851,)
Test data shape (203, 5) (203,)
['f1', 'f2', 'f3', 'f4', 'f5'] ['y']
(833, 6) (194, 6)
Min and max edge weight train: 0.0 8.58204416374
Min and max edge weight test: 0.0 8.82849412947
Model results:
MSE 2.50
Yesterday results:
MSE 2.36
Baseline results
Baseline: Mean Regressor
MSE 4.21
Baseline: Median Regressor
MSE 4.00
****************************************************************************************************
2
Weighted Model: True
weighted: True
weighted: True
Train data shape (851, 5) (851,)
Test data shape (203, 5) (203,)
['f1', 'f2', 'f3', 'f4', 'f5'] ['y']
(783, 6) (150, 6)
Min and max edge weight train: 0.0 8.58204416374
Min and max edge weight test: 0.0 8.82849412947
Model results:
MSE 3.34
Yesterday results:
MSE 4.09
Baseline results
Baseline: Mean Regressor
MSE 4.26
Baseline: Median Regressor
MSE 4.13
*********************************************************************************

In [13]:
weighted = False
for whichNetwork in [1,2,3]:
    print whichNetwork
    print "Weighted Model:", weighted
    X_train, y_train, X_test, y_test = makePairTrainTest(train_pairs, test_pairs, 
                                                         weekend_bluetooth_time_graphs, 
                                                         communication_time_graphs, 
                                                         weighted, whichNetwork)
    train_df, test_df, features_names, target_names = makeData(X_train, y_train, X_test, y_test)
    print features_names, target_names
    
    print train_df.shape, test_df.shape
    
    print "Train Distribution:", Counter(train_df.y)
    print "Test Distribution:", Counter(test_df.y)

    print "Model results:"
    fitClassModel(train_df[features_names], train_df[target_names], test_df[features_names], test_df[target_names])

    print "Train Distribution:", Counter(train_df.y)
    print "Test Distribution:", Counter(test_df.y)

    print "Yesterday results:"
    fitClassModel(train_df.f1.values.reshape(-1, 1), train_df[target_names], test_df.f1.values.reshape(-1, 1), test_df[target_names])

    print "Train Distribution:", Counter(train_df.y)
    print "Test Distribution:", Counter(test_df.y)

    print "Baseline results:"
    baselineClassifier(train_df[features_names], train_df.y, test_df[features_names], test_df.y)
    print "*"*100

1
Weighted Model: False
weighted: False
weighted: False
Train data shape (851, 5) (851,)
Test data shape (203, 5) (203,)
['f1', 'f2', 'f3', 'f4', 'f5'] ['y']
(827, 6) (193, 6)
Train Distribution: Counter({1: 534, 0: 293})
Test Distribution: Counter({1: 107, 0: 86})
Model results:
F1 0.76
Confusion matrix
[[65 21]
 [25 82]]
Train Distribution: Counter({1: 534, 0: 293})
Test Distribution: Counter({1: 107, 0: 86})
Yesterday results:
F1 0.75
Confusion matrix
[[63 23]
 [25 82]]
Train Distribution: Counter({1: 534, 0: 293})
Test Distribution: Counter({1: 107, 0: 86})
Baseline results:
Baseline: Random Classifier
F1 0.52
Confusion matrix
[[44 42]
 [50 57]]
****************************************************************************************************
2
Weighted Model: False
weighted: False
weighted: False
Train data shape (851, 5) (851,)
Test data shape (203, 5) (203,)
['f1', 'f2', 'f3', 'f4', 'f5'] ['y']
(726, 6) (131, 6)
Train Distribution: Counter({1: 463, 0: 263})
Test Distribution: 